<a href="https://colab.research.google.com/github/annamatias/dataengineer/blob/main/data_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIAP - Bootcamp Data Engineering
Anna Karoliny Matias dos Santos - RM355813

## Motivação
- Utilizei nesse arquivo ao invés de Pandas, uso o Pyspark, porque eu gosto de trabalhar com ele e acho mais fácil
> **Tive alguns problemas ao realizar a leitura direto do link, portanto eu realizei o import do arquivo no meu drive, para utilização**



# Outras Informações

*   cadastro.csv (dados de cadastro)
*   dados_medicos.csv (registros médicos)

Temos que unificar as duas bases e criar uma nova base agrupada

Código localizado em https://github.com/renzoziegler/data_engineering_bootcamp
*   Ler os arquivos CSV fornecidos
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/cadastro.csv
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/dados_medicos.csv
*   Realizar uma análise exploratória dos dados


# Instalação Pyspark

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark==3.1.2 # Installing pyspark matching the Spark version

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
  .master('local[*]')\
  .appName("bootcamp data eng")\
  .getOrCreate()

# Imports

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import to_date
from pyspark.sql.functions import date_format

In [4]:
!pip install fuzzywuzzy
import pandas as pd
from fuzzywuzzy import fuzz

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Leitura de Dados

## Cadastro

In [5]:
df1 = spark.read.format("csv").option("header", "true").load("/content/sample_data/cadastro.csv")

In [6]:
df1.show(5, False)

+-------------------------------+----------+-----------+----------------------+---------+---------------------------------------------------------------------+-------------+
|nome                           |data_nasc |cpf        |mae                   |celular  |email                                                                |telefone_fixo|
+-------------------------------+----------+-----------+----------------------+---------+---------------------------------------------------------------------+-------------+
|Cauã Isaac Caldeira            |02-03-1945|35083528118|Vitória Kamilly Sueli |989048340|cauaisaaccaldeira__cauaisaaccaldeira@hardquality.com.br              |28317237     |
|Lúcia Carolina Luzia Almada    |11-11-1946|6480417194 |Stefany Teresinha     |997275887|luciacarolinaluziaalmada_@santarte.com                               |27446850     |
|Vicente Marcos Edson Castro    |15-01-1959|72192158432|Cecília Laura Larissa |994560743|vicentemarcosedsoncastro-74@lavorosjc.com

## Dados Medicos

In [7]:
df2 = spark.read.format("csv").option("header", "true").load('/content/sample_data/dados_medicos.csv')

In [8]:
df2.show(5, False)

+---------------------------+----------+-----------+--------------+----+------+
|nome                       |data_nasc |cpf        |tipo_sanguineo|peso|altura|
+---------------------------+----------+-----------+--------------+----+------+
|Alice Liz Adriana Duarte   |22/10/1975|58548099313|O-            |89  |1,57  |
|Kaue César Lima            |01/01/1900|31515413217|AB-           |62  |1,99  |
|Otávio Felipe Henry Almeida|10/01/1951|58271711156|A-            |85  |1,66  |
|Rafaela Francisca Mendes   |07/11/1974|31087464641|O+            |46  |1,74  |
|Bryan Joaquim Roberto Rosa |07/09/1957|61606247174|AB-           |66  |1,67  |
+---------------------------+----------+-----------+--------------+----+------+
only showing top 5 rows



# Exercícios

## Exercício 2: Pré-processamento
*   Quais tarefas vocês listam como necessárias?
 *  Padronização de campos
 *  Substituir valores
 *  Retirar caracteres especiais
*   Listar e executar!

### Cadastro Pré-Processamento

In [9]:
df1.printSchema()

root
 |-- nome: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- mae: string (nullable = true)
 |-- celular: string (nullable = true)
 |-- email: string (nullable = true)
 |-- telefone_fixo: string (nullable = true)



In [10]:
df_cadastro = df1 \
    .withColumnRenamed("nome", "nome_completo") \
    .withColumnRenamed("data_nasc", "data_nascimento") \
    .withColumnRenamed("mae", "nome_mae") \
    .withColumnRenamed("telefone_fixo", "telefone_fixo")

In [11]:
df_cadastro.show(5, False)

+-------------------------------+---------------+-----------+----------------------+---------+---------------------------------------------------------------------+-------------+
|nome_completo                  |data_nascimento|cpf        |nome_mae              |celular  |email                                                                |telefone_fixo|
+-------------------------------+---------------+-----------+----------------------+---------+---------------------------------------------------------------------+-------------+
|Cauã Isaac Caldeira            |02-03-1945     |35083528118|Vitória Kamilly Sueli |989048340|cauaisaaccaldeira__cauaisaaccaldeira@hardquality.com.br              |28317237     |
|Lúcia Carolina Luzia Almada    |11-11-1946     |6480417194 |Stefany Teresinha     |997275887|luciacarolinaluziaalmada_@santarte.com                               |27446850     |
|Vicente Marcos Edson Castro    |15-01-1959     |72192158432|Cecília Laura Larissa |994560743|vicentemarc

#### Padronizar data de nascimento para o formato DD/MM/YYYY

In [12]:
df_cadastro = df_cadastro.withColumn("data_nascimento", to_date(col("data_nascimento"), "dd-MM-yyyy"))
df_cadastro = df_cadastro.withColumn("data_nascimento", date_format(col("data_nascimento"), "dd/MM/yyyy"))

#### Padronizar CPF para o formato 000.000.000-00

In [13]:
df_cadastro = df_cadastro.withColumn("cpf", lpad(col("cpf"), 11, "0"))  # Garantir 11 dígitos
df_cadastro = df_cadastro.withColumn("cpf", concat(
    col("cpf").substr(1, 3), lit("."),
    col("cpf").substr(4, 3), lit("."),
    col("cpf").substr(7, 3), lit("-"),
    col("cpf").substr(10, 2)
))

#### Padronizar números de celular e telefone fixo para o formato 00000-0000 (sem DDD)

In [14]:
df_cadastro = df_cadastro.withColumn("celular", lpad(col("celular"), 9, "0"))
df_cadastro = df_cadastro.withColumn("celular", concat(
    col("celular").substr(1, 5), lit("-"),
    col("celular").substr(6, 4)
))

df_cadastro = df_cadastro.withColumn("telefone_fixo", lpad(col("telefone_fixo"), 8, "0"))
df_cadastro = df_cadastro.withColumn("telefone_fixo", concat(
    col("telefone_fixo").substr(1, 4), lit("-"),
    col("telefone_fixo").substr(5, 4)
))

In [15]:
df_cadastro.show(5, False)

+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+
|nome_completo                  |data_nascimento|cpf           |nome_mae              |celular   |email                                                                |telefone_fixo|
+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+
|Cauã Isaac Caldeira            |02/03/1945     |350.835.281-18|Vitória Kamilly Sueli |98904-8340|cauaisaaccaldeira__cauaisaaccaldeira@hardquality.com.br              |2831-7237    |
|Lúcia Carolina Luzia Almada    |11/11/1946     |064.804.171-94|Stefany Teresinha     |99727-5887|luciacarolinaluziaalmada_@santarte.com                               |2744-6850    |
|Vicente Marcos Edson Castro    |15/01/1959     |721.921.584-32|Cecília Laura Larissa

In [16]:
df_cadastro.printSchema()

root
 |-- nome_completo: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- nome_mae: string (nullable = true)
 |-- celular: string (nullable = true)
 |-- email: string (nullable = true)
 |-- telefone_fixo: string (nullable = true)



### Dados Medicos Pré-Processamento

In [17]:
df2.printSchema()
df2.show(5, False)

root
 |-- nome: string (nullable = true)
 |-- data_nasc: string (nullable = true)
 |-- cpf: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- peso: string (nullable = true)
 |-- altura: string (nullable = true)

+---------------------------+----------+-----------+--------------+----+------+
|nome                       |data_nasc |cpf        |tipo_sanguineo|peso|altura|
+---------------------------+----------+-----------+--------------+----+------+
|Alice Liz Adriana Duarte   |22/10/1975|58548099313|O-            |89  |1,57  |
|Kaue César Lima            |01/01/1900|31515413217|AB-           |62  |1,99  |
|Otávio Felipe Henry Almeida|10/01/1951|58271711156|A-            |85  |1,66  |
|Rafaela Francisca Mendes   |07/11/1974|31087464641|O+            |46  |1,74  |
|Bryan Joaquim Roberto Rosa |07/09/1957|61606247174|AB-           |66  |1,67  |
+---------------------------+----------+-----------+--------------+----+------+
only showing top 5 rows



In [18]:
df_dados_medicos = df2 \
    .withColumnRenamed("nome", "nome_completo_dados_medicos") \
    .withColumnRenamed("data_nasc", "data_nascimento_dados_medicos")\
    .withColumnRenamed("cpf", "cpf_dados_medicos")

#### Padronizar data de nascimento para o formato DD/MM/YYYY

In [19]:
df_dados_medicos = df_dados_medicos.withColumn("data_nascimento_dados_medicos", to_date(col("data_nascimento_dados_medicos"), "dd/MM/yyyy"))
df_dados_medicos = df_dados_medicos.withColumn("data_nascimento_dados_medicos", date_format(col("data_nascimento_dados_medicos"), "dd/MM/yyyy"))

#### Padronizar CPF para o formato 000.000.000-00

In [20]:
df_dados_medicos = df_dados_medicos.withColumn("cpf_dados_medicos", lpad(col("cpf_dados_medicos"), 11, "0"))  # Garantir 11 dígitos
df_dados_medicos = df_dados_medicos.withColumn("cpf_dados_medicos", concat(
    col("cpf_dados_medicos").substr(1, 3), lit("."),
    col("cpf_dados_medicos").substr(4, 3), lit("."),
    col("cpf_dados_medicos").substr(7, 3), lit("-"),
    col("cpf_dados_medicos").substr(10, 2)
))

#### Padronizar peso e altura

In [21]:
# Remover possíveis caracteres especiais (como vírgulas) e converter para float
df_dados_medicos = df_dados_medicos.withColumn("peso", regexp_replace(col("peso"), ",", ".").cast("float"))
df_dados_medicos = df_dados_medicos.withColumn("altura", regexp_replace(col("altura"), ",", ".").cast("float"))


In [22]:
df_dados_medicos.printSchema()

root
 |-- nome_completo_dados_medicos: string (nullable = true)
 |-- data_nascimento_dados_medicos: string (nullable = true)
 |-- cpf_dados_medicos: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- peso: float (nullable = true)
 |-- altura: float (nullable = true)



In [23]:
df_dados_medicos.show(5, False)

+---------------------------+-----------------------------+-----------------+--------------+----+------+
|nome_completo_dados_medicos|data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|
+---------------------------+-----------------------------+-----------------+--------------+----+------+
|Alice Liz Adriana Duarte   |22/10/1975                   |585.480.993-13   |O-            |89.0|1.57  |
|Kaue César Lima            |01/01/1900                   |315.154.132-17   |AB-           |62.0|1.99  |
|Otávio Felipe Henry Almeida|10/01/1951                   |582.717.111-56   |A-            |85.0|1.66  |
|Rafaela Francisca Mendes   |07/11/1974                   |310.874.646-41   |O+            |46.0|1.74  |
|Bryan Joaquim Roberto Rosa |07/09/1957                   |616.062.471-74   |AB-           |66.0|1.67  |
+---------------------------+-----------------------------+-----------------+--------------+----+------+
only showing top 5 rows



## Exercício 3: Indexação
*    Listar possíveis blocking Keys
*    Definir uma a ser utilizada
*    Criar os blocos
*    Listar os possíveis matches em cada bloco

In [24]:
df_cadastro = df_cadastro.withColumn(
    "data_nascimento",
    to_date(col("data_nascimento"), "dd/MM/yyyy")
).withColumn("nasc_mes", month("data_nascimento"))

df_dados_medicos = df_dados_medicos.withColumn(
    "data_nascimento_dados_medicos",
    to_date(col("data_nascimento_dados_medicos"), "dd/MM/yyyy")
).withColumn("nasc_mes", month("data_nascimento_dados_medicos"))

#### Extrair mês de nascimento e criar colunas auxiliares

In [25]:
df_cadastro = df_cadastro.withColumn("nasc_mes", month("data_nascimento"))
df_dados_medicos = df_dados_medicos.withColumn("nasc_mes", month("data_nascimento_dados_medicos"))

In [26]:
df_cadastro.show(5, False)
df_dados_medicos.show(5, False)

+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+
|nome_completo                  |data_nascimento|cpf           |nome_mae              |celular   |email                                                                |telefone_fixo|nasc_mes|
+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+
|Cauã Isaac Caldeira            |1945-03-02     |350.835.281-18|Vitória Kamilly Sueli |98904-8340|cauaisaaccaldeira__cauaisaaccaldeira@hardquality.com.br              |2831-7237    |3       |
|Lúcia Carolina Luzia Almada    |1946-11-11     |064.804.171-94|Stefany Teresinha     |99727-5887|luciacarolinaluziaalmada_@santarte.com                               |2744-6850    |11      |
|Vicente Marcos Edson Castro    |1959-01

#### Contar pessoas por mês de nascimento em ambos os DataFrames

In [27]:
cadastro_nasc_mes = df_cadastro.groupBy("nasc_mes").count().withColumnRenamed("count", "cadastro")
dados_medicos_nasc_mes = df_dados_medicos.groupBy("nasc_mes").count().withColumnRenamed("count", "dados_medicos")

In [28]:
cadastro_nasc_mes.show()
dados_medicos_nasc_mes.show()

+--------+--------+
|nasc_mes|cadastro|
+--------+--------+
|      12|      24|
|       1|      34|
|       6|      30|
|       3|      37|
|       5|      42|
|       9|      28|
|       4|      37|
|       8|      40|
|       7|      19|
|      10|      32|
|      11|      33|
|       2|      37|
+--------+--------+

+--------+-------------+
|nasc_mes|dados_medicos|
+--------+-------------+
|      12|           19|
|    null|            3|
|       1|           93|
|       6|           22|
|       3|           29|
|       5|           25|
|       9|           21|
|       4|           25|
|       8|           32|
|       7|           15|
|      10|           31|
|      11|           25|
|       2|           28|
+--------+-------------+



#### Unir os dois DataFrames para blocos mensais

In [29]:
blocos = cadastro_nasc_mes.join(dados_medicos_nasc_mes, "nasc_mes", "outer").fillna(0)

In [30]:
blocos.show()

+--------+--------+-------------+
|nasc_mes|cadastro|dados_medicos|
+--------+--------+-------------+
|      12|      24|           19|
|       0|       0|            3|
|       1|      34|           93|
|       6|      30|           22|
|       3|      37|           29|
|       5|      42|           25|
|       9|      28|           21|
|       4|      37|           25|
|       8|      40|           32|
|       7|      19|           15|
|      10|      32|           31|
|      11|      33|           25|
|       2|      37|           28|
+--------+--------+-------------+



#### Calcular o produto entre as colunas 'cadastro' e 'dados_medicos' e somar

In [31]:
produto_nasc_mes = blocos.select((col("cadastro") * col("dados_medicos")).alias("produto"))
soma_produto_nasc_mes = produto_nasc_mes.groupBy().sum("produto").first()[0]
print("Soma dos produtos por mês de nascimento:", soma_produto_nasc_mes)

Soma dos produtos por mês de nascimento: 12332


#### Extraindo o primeiro dígito do CPF

In [32]:
df_cadastro = df_cadastro.withColumn("cpf_0", substring("cpf", 1, 1))
df_dados_medicos = df_dados_medicos.withColumn("cpf_0", substring("cpf_dados_medicos", 1, 1))

In [33]:
df_cadastro.show(5, False)
df_dados_medicos.show(5, False)

+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+
|nome_completo                  |data_nascimento|cpf           |nome_mae              |celular   |email                                                                |telefone_fixo|nasc_mes|cpf_0|
+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+
|Cauã Isaac Caldeira            |1945-03-02     |350.835.281-18|Vitória Kamilly Sueli |98904-8340|cauaisaaccaldeira__cauaisaaccaldeira@hardquality.com.br              |2831-7237    |3       |3    |
|Lúcia Carolina Luzia Almada    |1946-11-11     |064.804.171-94|Stefany Teresinha     |99727-5887|luciacarolinaluziaalmada_@santarte.com                               |2744-6850    |11      |0    |
|Vicente M

#### Contagem por primeiro dígito do CPF

In [34]:
cadastro_cpf_0 = df_cadastro.groupBy("cpf_0").count().withColumnRenamed("count", "cadastro")
dados_medicos_cpf_0 = df_dados_medicos.groupBy("cpf_0").count().withColumnRenamed("count", "dados_medicos")

In [35]:
cadastro_cpf_0.show()
dados_medicos_cpf_0.show()

+-----+--------+
|cpf_0|cadastro|
+-----+--------+
|    7|      48|
|    3|      44|
|    8|      32|
|    0|      37|
|    5|      50|
|    6|      40|
|    9|      33|
|    1|      30|
|    4|      41|
|    2|      38|
+-----+--------+

+-----+-------------+
|cpf_0|dados_medicos|
+-----+-------------+
|    7|           40|
|    3|           34|
|    8|           20|
|    0|           96|
|    5|           41|
|    6|           31|
|    9|           22|
|    1|           24|
|    4|           34|
|    2|           26|
+-----+-------------+



#### Criando blocos por primeiro dígito do CPF

In [36]:
blocos_cpf = cadastro_cpf_0.join(dados_medicos_cpf_0, "cpf_0", "outer").fillna(0)
produto_cpf_0 = blocos_cpf.select((col("cadastro") * col("dados_medicos")).alias("produto"))
soma_produto_cpf_0 = produto_cpf_0.groupBy().sum("produto").first()[0]
print("Soma dos produtos por primeiro dígito do CPF:", soma_produto_cpf_0)

Soma dos produtos por primeiro dígito do CPF: 14726


#### Separar dados em blocos de 0 a 9 usando o primeiro dígito do CPF

In [37]:
cadastro_blocos = {str(i): df_cadastro.filter(col("cpf_0") == str(i)) for i in range(10)}
dados_medicos_blocos = {str(i): df_dados_medicos.filter(col("cpf_0") == str(i)) for i in range(10)}


##### Verificando execução em blocos para Cadastro e Dados medicos

In [38]:
for i in range(10):
  print(cadastro_blocos[f"{i}"].show(2, False))

+---------------------------------+---------------+--------------+------------------+----------+--------------------------------------------------------------------------------+-------------+--------+-----+
|nome_completo                    |data_nascimento|cpf           |nome_mae          |celular   |email                                                                           |telefone_fixo|nasc_mes|cpf_0|
+---------------------------------+---------------+--------------+------------------+----------+--------------------------------------------------------------------------------+-------------+--------+-----+
|Lúcia Carolina Luzia Almada      |1946-11-11     |064.804.171-94|Stefany Teresinha |99727-5887|luciacarolinaluziaalmada_@santarte.com                                          |2744-6850    |11      |0    |
|Sabrina Aparecida Mirella Ribeiro|1993-05-17     |067.127.576-30|Cláudia Sophia    |99180-6203|sabrinaaparecidamirellaribeiro__sabrinaaparecidamirellaribeiro@wwlimpador.co

In [39]:
for i in range(10):
  print(dados_medicos_blocos[f"{i}"].show(2, False))

+---------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+
|nome_completo_dados_medicos|data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|
+---------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+
|Benício Iago Silveira      |1900-01-01                   |000.000.000-00   |O+            |97.0|1.72  |1       |0    |
|Lucca Carlos Ferreira      |1900-01-01                   |033.608.164-20   |O-            |99.0|1.74  |1       |0    |
+---------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+
only showing top 2 rows

None
+------------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+
|nome_completo_dados_medicos   |data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|
+---

In [40]:
cadastro_blocos['0'].show(2, truncate=False)
dados_medicos_blocos['0'].show(2, truncate=False)

+---------------------------------+---------------+--------------+------------------+----------+--------------------------------------------------------------------------------+-------------+--------+-----+
|nome_completo                    |data_nascimento|cpf           |nome_mae          |celular   |email                                                                           |telefone_fixo|nasc_mes|cpf_0|
+---------------------------------+---------------+--------------+------------------+----------+--------------------------------------------------------------------------------+-------------+--------+-----+
|Lúcia Carolina Luzia Almada      |1946-11-11     |064.804.171-94|Stefany Teresinha |99727-5887|luciacarolinaluziaalmada_@santarte.com                                          |2744-6850    |11      |0    |
|Sabrina Aparecida Mirella Ribeiro|1993-05-17     |067.127.576-30|Cláudia Sophia    |99180-6203|sabrinaaparecidamirellaribeiro__sabrinaaparecidamirellaribeiro@wwlimpador.co

### Join entre informações

In [41]:
df_join = df_cadastro.join(df_dados_medicos, df_dados_medicos["cpf_dados_medicos"] == df_cadastro["cpf"], "inner") #.drop("nome_completo_dados_medicos", "data_nascimento_dados_medicos")

In [42]:
df_join.show(5, False)

+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+------------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+
|nome_completo                  |data_nascimento|cpf           |nome_mae              |celular   |email                                                                |telefone_fixo|nasc_mes|cpf_0|nome_completo_dados_medicos   |data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|
+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+------------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+
|Cauã Isaac Caldeira            |1945-03-02

## Exercício 4: Comparação par a par
*    Desenvolver um algoritmo de comparação de cada match potencial do exercício anterior
*    Definir uma nota de similaridade para cada comparação
 *   Para cada parâmetro

#### Função de similaridade para nomes usando fuzzy matching

In [43]:
def nome_similarity(nome1, nome2):
    if nome1 == nome2:
        return 1.0
    return fuzz.token_sort_ratio(nome1, nome2) / 100

# Registrar a função como UDF
nome_similarity_udf = udf(nome_similarity, FloatType())

#### Função de comparação para CPF e Data de Nascimento

In [44]:
def comparaRegistros(dados_medicos, cadastro):
    # Comparação de CPF (1 se igual, 0 caso contrário)
    cpf_match = when(dados_medicos["cpf_dados_medicos"] == cadastro["cpf"], 1).otherwise(0)

    # Comparação de Data de Nascimento exata ou com troca de mês e dia
    data_nasc_match = when(
        (dados_medicos["data_nascimento_dados_medicos"] == cadastro["data_nascimento"]) |
        ((dados_medicos["nasc_mes"] == cadastro["nasc_mes"])), 1
    ).otherwise(0)

    # Comparação de Nome (similaridade usando a função UDF)
    nome_match = nome_similarity_udf(dados_medicos["nome_completo_dados_medicos"], cadastro["nome_completo"])

    return cpf_match, nome_match, data_nasc_match

In [45]:
df_join.show(5, False)

+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+------------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+
|nome_completo                  |data_nascimento|cpf           |nome_mae              |celular   |email                                                                |telefone_fixo|nasc_mes|cpf_0|nome_completo_dados_medicos   |data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|
+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+------------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+
|Cauã Isaac Caldeira            |1945-03-02

#### Adicionando pares de similaridade com Nome, CPF, Data de nascimento e soma

In [46]:
# Para cada bloco de CPF
pares = []

for i in range(10):
    print(f"CPF começando com {i}")

    dados_medicos_df = dados_medicos_blocos[f"{i}"]
    cadastro_df = cadastro_blocos[f"{i}"]

    dados_medicos_df.printSchema()
    cadastro_df.printSchema()

    cpf_match, nome_match, data_nasc_match = comparaRegistros(dados_medicos_df, cadastro_df)

    # Adicionar colunas de similaridade e somar para calcular simSum
    comparacao_df = df_join \
        .withColumn("cpf_similaridade", cpf_match) \
        .withColumn("nome_similaridade", nome_match) \
        .withColumn("data_nasc_similaridade", data_nasc_match) \
        .withColumn("simSum", col("cpf_similaridade") + col("data_nasc_similaridade") + col("nome_similaridade"))

    # Adicionar ao resultado final
    pares.append(comparacao_df)

CPF começando com 0
root
 |-- nome_completo_dados_medicos: string (nullable = true)
 |-- data_nascimento_dados_medicos: date (nullable = true)
 |-- cpf_dados_medicos: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-- peso: float (nullable = true)
 |-- altura: float (nullable = true)
 |-- nasc_mes: integer (nullable = true)
 |-- cpf_0: string (nullable = true)

root
 |-- nome_completo: string (nullable = true)
 |-- data_nascimento: date (nullable = true)
 |-- cpf: string (nullable = true)
 |-- nome_mae: string (nullable = true)
 |-- celular: string (nullable = true)
 |-- email: string (nullable = true)
 |-- telefone_fixo: string (nullable = true)
 |-- nasc_mes: integer (nullable = true)
 |-- cpf_0: string (nullable = true)

CPF começando com 1
root
 |-- nome_completo_dados_medicos: string (nullable = true)
 |-- data_nascimento_dados_medicos: date (nullable = true)
 |-- cpf_dados_medicos: string (nullable = true)
 |-- tipo_sanguineo: string (nullable = true)
 |-

#### Unir todos os blocos em um único DataFrame e Filtro por tipo de similaridade

In [47]:
pares_df = pares[0]
for p in pares[1:]:
    pares_df = pares_df.union(p)

# Filtrar os DataFrames por tipo de similaridade
matches = pares_df.filter(pares_df.simSum >= 2.5)
potenciais = pares_df.filter((pares_df.simSum < 2.5) & (pares_df.simSum >= 1.5))
non_matches = pares_df.filter(pares_df.simSum < 1.5)


print("Matches:")
matches.show(2, False)

print("\nPotenciais:")
potenciais.show(2, False)

print("\nNon-matches:")
non_matches.show(2, False)

Matches:
+---------------------------+---------------+--------------+----------------------+----------+-------------------------------------------------------+-------------+--------+-----+---------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+----------------+-----------------+----------------------+------+
|nome_completo              |data_nascimento|cpf           |nome_mae              |celular   |email                                                  |telefone_fixo|nasc_mes|cpf_0|nome_completo_dados_medicos|data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|cpf_similaridade|nome_similaridade|data_nasc_similaridade|simSum|
+---------------------------+---------------+--------------+----------------------+----------+-------------------------------------------------------+-------------+--------+-----+---------------------------+-----------------------------+-----------------+----------

## Exercício 5: Classificação
*    Classificar cada par, baseado num threshold
*    Definir um peso para cada parâmetro

#### Calculando a soma de similaridades (simSum) e adicionando categorias de matches, potenciais e non-matches

In [48]:
pares_df2 = pares_df.withColumn(
    "simSum", col("nome_similaridade") + col("cpf_similaridade") + col("data_nasc_similaridade")
)

In [49]:
pares_df2.show(5, False)

+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+------------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+----------------+-----------------+----------------------+------+
|nome_completo                  |data_nascimento|cpf           |nome_mae              |celular   |email                                                                |telefone_fixo|nasc_mes|cpf_0|nome_completo_dados_medicos   |data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|cpf_similaridade|nome_similaridade|data_nasc_similaridade|simSum|
+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+------------------------------+----

#### Classificando os registros com base na simSum

In [50]:
pares_df2 = pares_df2.withColumn(
    "categoria",
    when(col("simSum") == 3, "matches")
    .when((col("simSum") >= 2) & (col("simSum") < 3), "potenciais")
    .otherwise("non_matches")
)

In [51]:
pares_df2.show(5, False)

+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+------------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+----------------+-----------------+----------------------+------+----------+
|nome_completo                  |data_nascimento|cpf           |nome_mae              |celular   |email                                                                |telefone_fixo|nasc_mes|cpf_0|nome_completo_dados_medicos   |data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|cpf_similaridade|nome_similaridade|data_nasc_similaridade|simSum|categoria |
+-------------------------------+---------------+--------------+----------------------+----------+---------------------------------------------------------------------+-------------+--------+-----+-------------

#### Separando cada categoria em DataFrames distintos

In [52]:
matches = pares_df2.filter(col("categoria") == "matches")
potenciais = pares_df2.filter(col("categoria") == "potenciais")
non_matches = pares_df2.filter(col("categoria") == "non_matches")

In [53]:
matches.show(2, False)
potenciais.show(2, False)
non_matches.show(2, False)

+---------------------------+---------------+--------------+----------------------+----------+-------------------------------------------------------+-------------+--------+-----+---------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+----------------+-----------------+----------------------+------+---------+
|nome_completo              |data_nascimento|cpf           |nome_mae              |celular   |email                                                  |telefone_fixo|nasc_mes|cpf_0|nome_completo_dados_medicos|data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|cpf_similaridade|nome_similaridade|data_nasc_similaridade|simSum|categoria|
+---------------------------+---------------+--------------+----------------------+----------+-------------------------------------------------------+-------------+--------+-----+---------------------------+-----------------------------+-----------------

#### Encontrando registros de dados_medicos que estão ausentes em matches

In [54]:
matches_registros = matches.filter(col("categoria") == "matches")  # Colunas de registro nos matches
is_missing = dados_medicos_df.join(
    matches_registros,
    on=["nome_completo_dados_medicos", "cpf_dados_medicos", "data_nascimento_dados_medicos"],
    how="left_anti"
).select("nome_completo_dados_medicos", "data_nascimento_dados_medicos", "cpf_dados_medicos")

In [55]:
is_missing.count()

7

#### Exibindo uma amostra de potenciais com simSum == 2 para avaliação manual

In [56]:
potenciais_sample = potenciais.filter(col("simSum") == 2).sample(0.1)  # 10% da amostra
potenciais_sample.count()

32

#### Exibindo uma amostra de non_matches com simSum > 1.5 para avaliação manual

In [57]:
non_matches_sample = non_matches.filter(col("simSum") > 1.5).sample(0.1)
non_matches_sample.count()

23

## Exercício 6: Avaliação
*     Buscar uma amostra das classificações (de 10 a 20 registros)
*     Classificar cada uma em TP, TN, FP, FN
*     Calcular acurácia, precisão e recall

In [58]:
# Definir variáveis True Positive (TP), False Positive (FP), True Negative (TN), False Negative (FN) para métricas de avaliação
TP = 10
FP = 0
TN = 2
FN = 8

# Calcular métricas de avaliação
acuracia = (TP + TN) / (TP + FP + TN + FN)
precisao = TP / (TP + FP) if TP + FP != 0 else 0
recall = TP / (TP + FN) if TP + FN != 0 else 0
fmeasure = (2 * precisao * recall) / (precisao + recall) if precisao + recall != 0 else 0

# Exibir as métricas
print("Acuracia: {:.2f}".format(acuracia))
print("Recall: {:.2f}".format(recall))
print("Precisão: {:.2f}".format(precisao))
print("F-measure: {:.2f}".format(fmeasure))

# Exibir as amostras para verificação
potenciais_sample.show(5, truncate=False)
non_matches_sample.show(5, truncate=False)

Acuracia: 0.60
Recall: 0.56
Precisão: 1.00
F-measure: 0.71
+------------------------------+---------------+--------------+-----------------------+----------+----------------------------------------------------------------------------+-------------+--------+-----+------------------------------+-----------------------------+-----------------+--------------+----+------+--------+-----+----------------+-----------------+----------------------+------+----------+
|nome_completo                 |data_nascimento|cpf           |nome_mae               |celular   |email                                                                       |telefone_fixo|nasc_mes|cpf_0|nome_completo_dados_medicos   |data_nascimento_dados_medicos|cpf_dados_medicos|tipo_sanguineo|peso|altura|nasc_mes|cpf_0|cpf_similaridade|nome_similaridade|data_nasc_similaridade|simSum|categoria |
+------------------------------+---------------+--------------+-----------------------+----------+---------------------------------------

## Exercício 7: Revisão Manual (Clerical Review)

Com a classificação feita, que alterações nos valores de threshold poderíamos fazer, para minimizar os matches potenciais e melhorar a avaliação do modelo??

### Resposta

Na minha opinião, eu utilizei do join entre as duas bases para facilitar a união e realizar a classificação depois, vejo que isso ajudou muito, visto que antes tinhamos separado bloco por bloco e conseguimos ver que a regressão que fizemos ela foi longa e tivemos um resultado muito maior de combinações.

Outro ponto que devemos classificar e melhorar a avaliação do modelo, são as possibilidades, temos sempre que avaliar o que iremos utilizar do match, se é cpf, mês, o nome (no caso do nome, temos muitas variações, mas podemos utilizar, depende de como, teriamos que refinar e analisar colocando caso a caso para sermos mais assertivos). Então diria isso, além das indexações, padronizações, sempre avaliar melhor o que iremos utilizar como classificação, se é bom o suficiente para termos alterações mais assertivas nos valores do threshold.